[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openvinotoolkit/openvino_tensorflow/blob/master/docs/cloud_instructions/Colab_OpenVINO_TensorFlow_classification_example.ipynb)

## INSTALL OpenVINO™ integration with TensorFlow


In [1]:
# Upload the required wheel files, models and images in a google drive folder
# Uncomment and run the below command to copy them in your current workspace
#!cp /content/drive/MyDrive/TF-OV/working_dir_files/* . 

!pip -q install --upgrade pip
!pip -q install pillow

# Install stock TensorFlow
!pip -q install tensorflow==2.6.0 

# Install OpenVINO™ integration with TensorFlow
!pip -q install openvino-tensorflow

     |████████████████████████████████| 454.4 MB 9.4 kB/s             
     |████████████████████████████████| 4.0 MB 42.8 MB/s            
     |████████████████████████████████| 1.2 MB 77.8 MB/s            
     |████████████████████████████████| 462 kB 58.7 MB/s            
     |████████████████████████████████| 26.5 MB 19 kB/s             


In [2]:
!ldd --version
!python3 --version

ldd (Ubuntu GLIBC 2.27-3ubuntu1.3) 2.27
Copyright (C) 2018 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
Written by Roland McGrath and Ulrich Drepper.
Python 3.7.12


In [3]:
!git clone --quiet https://github.com/openvinotoolkit/openvino_tensorflow.git
%cd openvino_tensorflow
!git submodule init
!git submodule update --recursive
%cd ..


/content/openvino_tensorflow
Submodule 'ocm' (https://github.com/intel/ocm) registered for path 'ocm'
Cloning into '/content/openvino_tensorflow/ocm'...
Submodule path 'ocm': checked out '1536f9877b8817872aa0e96859ac6547c1850c96'
/content


# Lets get the model

In [4]:
!curl -L "https://storage.googleapis.com/download.tensorflow.org/models/inception_v3_2016_08_28_frozen.pb.tar.gz" | tar -C openvino_tensorflow/examples/data -xz 


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 84.5M  100 84.5M    0     0  49.9M      0  0:00:01  0:00:01 --:--:-- 49.9M


# Now lets infer

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import backend as K
from IPython.display import HTML

import tensorflow as tf
import numpy as np
import time
import argparse
import matplotlib.pyplot as plt
import os
import openvino_tensorflow as ovtf
from subprocess import check_output, call


In [6]:
def load_graph(model_file):
    """Parse and load model from the protobuf binary(pb) file

    Parameters:
    model_file (string): Path of the model pb file

    Returns:
    graph: TF graph object for the parsed model
    """  
    graph = tf.Graph()
    graph_def = tf.compat.v1.GraphDef()

    assert os.path.exists(model_file), "Could not find directory"
    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

In [7]:

def read_tensor_from_image_file(file_name,
                                input_height=299,
                                input_width=299,
                                input_mean=0,
                                input_std=255):
    """Reads input image file, resize it to given input height and width.
       Normalize the image using input mean and standard deviation and returns the image tensor
    """  
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.io.read_file(file_name, input_name)
    if file_name.endswith(".png"):
        image_reader = tf.image.decode_png(
            file_reader, channels=3, name="png_reader")
    elif file_name.endswith(".gif"):
        image_reader = tf.squeeze(
            tf.image.decode_gif(file_reader, name="gif_reader"))
    elif file_name.endswith(".bmp"):
        image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
    else:
        image_reader = tf.image.decode_jpeg(
            file_reader, channels=3, name="jpeg_reader")
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0)
    resized = tf.compat.v1.image.resize_bilinear(dims_expander,
                                                 [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.compat.v1.Session()
    result = sess.run(normalized)

    return result

In [8]:
def load_labels(label_file):
    """Parses the label file, assuming that labels are separated with a newline
       in the file and returns the list of labels.
    """  
    label = []
    proto_as_ascii_lines = tf.io.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

In [9]:
def infer_openvino_tensorflow(model_file, input_layer, output_layer, file_name , input_height, input_width, input_mean, input_std, label_file):
    """Takes the tensorflow model and all other input parameters as arguments. 
       Run inference on the classification model and prints the predictions.
    """
    print("CREATE MODEL - BEGIN")
    graph = load_graph(model_file)
    print("CREATE MODEL - END")

    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    input_operation = graph.get_operation_by_name(input_name)
    output_operation = graph.get_operation_by_name(output_name)


    # update config params for openvino tensorflow
    config = tf.compat.v1.ConfigProto()
    config_ngraph_enabled = ovtf.update_config(config)

    print("PREDICTION - BEGIN") 

    with tf.compat.v1.Session(
            graph=graph, config=config_ngraph_enabled) as sess:
        t = read_tensor_from_image_file(
            file_name,
            input_height=input_height,
            input_width=input_width,
            input_mean=input_mean,
            input_std=input_std)
        # Warmup
        results = sess.run(output_operation.outputs[0],
                           {input_operation.outputs[0]: t})
        # Run
        
        for num_times in range(10):
            start = time.time()
            results = sess.run(output_operation.outputs[0],
                               {input_operation.outputs[0]: t})
            elapsed = time.time() - start
            print('Inference time in ms: %f' % (elapsed * 1000))
            
    print("PREDICTION - END")
    results = np.squeeze(results)

    if label_file:
        top_k = results.argsort()[-5:][::-1]
        labels = load_labels(label_file)
        for i in top_k:
            print(labels[i], results[i])
    else:
        print("No label file provided. Cannot print classification results")

In [10]:
%cd openvino_tensorflow/examples/data/

/content/openvino_tensorflow/examples/data


In [11]:
file_name = "grace_hopper.jpg"
model_file = "inception_v3_2016_08_28_frozen.pb"
label_file = "imagenet_slim_labels.txt"
input_height = 299
input_width = 299
input_mean = 0
input_std = 255
input_layer = "input"
output_layer = "InceptionV3/Predictions/Reshape_1"
backend_name = "CPU"
    

#Print list of available backends
print('Available Backends:')
backends_list = ovtf.list_backends()
for backend in backends_list:
    print(backend)
ovtf.set_backend(backend_name)

print("OpenVINO TensorFlow is enabled")
infer_openvino_tensorflow(model_file, input_layer, output_layer, file_name, input_height, input_width, input_mean, input_std, label_file )



Available Backends:
CPU
OpenVINO TensorFlow is enabled
CREATE MODEL - BEGIN
CREATE MODEL - END
PREDICTION - BEGIN
Inference time in ms: 180.995464
Inference time in ms: 181.902409
Inference time in ms: 197.049379
Inference time in ms: 193.738699
Inference time in ms: 191.765070
Inference time in ms: 191.222191
Inference time in ms: 185.688019
Inference time in ms: 191.789865
Inference time in ms: 199.031115
Inference time in ms: 186.476707
PREDICTION - END
military uniform 0.8343049
mortarboard 0.021869816
academic gown 0.010358169
pickelhaube 0.008008191
bulletproof vest 0.0053508924


In [12]:
#Disable
ovtf.disable()
print("")
print("")
print("OpenVINO TensorFlow is disabled")
infer_openvino_tensorflow(model_file, input_layer, output_layer, file_name, input_height, input_width, input_mean, input_std, label_file )
ovtf.enable()




OpenVINO TensorFlow is disabled
CREATE MODEL - BEGIN
CREATE MODEL - END
PREDICTION - BEGIN
Inference time in ms: 257.229567
Inference time in ms: 239.981890
Inference time in ms: 238.631248
Inference time in ms: 242.352962
Inference time in ms: 263.074160
Inference time in ms: 241.921186
Inference time in ms: 238.617659
Inference time in ms: 245.656729
Inference time in ms: 248.639584
Inference time in ms: 238.406658
PREDICTION - END
military uniform 0.8343058
mortarboard 0.02186954
academic gown 0.010358081
pickelhaube 0.0080081895
bulletproof vest 0.0053509064
